In [1]:
library(tidyverse)
library(Seurat)

source('milo_cellbender.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   0.3.4
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




In [2]:
other_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_other_01t') %>% 
    set_labels_to_class %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
other_clusters

[1] "other"

In [3]:
neuron_clusters = qs::qread('../00_cellbender/_targets/objects/obj_cb_neuron_00') %>% 
    set_labels_to_class %>%
    set_batch_to_lane %>%    
    prep_obj_for_milo_cb_v01 %>%
    `[[` %>%
    pull(labels) %>% 
    unique
neuron_clusters 

[1] "neuron"

In [4]:
day_splits = tibble(object = rlang::syms(c("exp_labelled_other", "exp_labelled_neuron")))
days = tibble(day = c("Day5", "Day14", "all"))
# days = tibble(day = c("Day14"))
day_splits = tidyr::crossing(day_splits, days)
# day_splits = day_splits %>% rowwise %>% mutate(day_label)
day_splits

object,day
<list>,<chr>
exp_labelled_other,all
exp_labelled_other,Day14
exp_labelled_other,Day5
exp_labelled_neuron,all
exp_labelled_neuron,Day14
exp_labelled_neuron,Day5


In [5]:
other_clusters_tibble = tibble(label = other_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'other'))


neuron_clusters_tibble = tibble(label = neuron_clusters) %>% 
tidyr::crossing(day_splits) %>%
filter(str_detect(as.character(object), 'neuron'))

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
clusters_tibble = clusters_tibble %>% 
mutate(cluster = label) %>%
rowwise %>%
mutate(label = case_when(day != "all" ~ paste0(c(day, as.character(label)), collapse='.'),
                          TRUE ~ paste0(label, collapse='.'))) %>%
filter(!str_detect(cluster, 'unassigned'))
clusters_tibble

label,object,day,cluster
<chr>,<list>,<chr>,<chr>
other,exp_labelled_other,all,other
Day14.other,exp_labelled_other,Day14,other
Day5.other,exp_labelled_other,Day5,other
neuron,exp_labelled_neuron,all,neuron
Day14.neuron,exp_labelled_neuron,Day14,neuron
Day5.neuron,exp_labelled_neuron,Day5,neuron


In [6]:
qs::qsave(clusters_tibble, 'clusters_tibble_class.qs')

In [7]:
clusters_tibble %>%
distinct(cluster) %>% pull(cluster) %>%
paste0('milo_', .) %>% dput

c("milo_other", "milo_neuron")
